# Hands-On-AI-In-Cyber
**SMS: Spam or Ham (Beginner)** 

Credits: https://www.kaggle.com/code/dejavu23/sms-spam-or-ham-beginner/notebook

Ziel: Klassifizierung von SMS texten zu Ham oder Spam

Behandelte Themen:

* **Removing Punctuation and Stopwords**
* **Tokenizer, Bag of words**  
* **Term frequency inverse document frequency (TFIDF)**

Anhand von diesem Datenset wird ein Naive Bayes model getestet:

* **Naive Bayes Classifier**


## Imports

In [1]:
import numpy as np 
import pandas as pd 

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.preprocessing import MinMaxScaler

from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
import os

## Functions

In [2]:
def plot_confusion_matrix(y_true, y_pred):
    mtx = confusion_matrix(y_true, y_pred)
    #fig, ax = plt.subplots(figsize=(4,4))
    sns.heatmap(mtx, annot=True, fmt='d', linewidths=.5,  
                cmap="Blues", cbar=False, ax=ax)
    #  square=True,
    plt.ylabel('true label')
    plt.xlabel('predicted label')

# **Part 1: Dataset insights**

In [3]:
data = pd.read_csv("data/spamHamDataSet.csv",encoding='latin-1')
#cleaning data
data['spam'] = data['label'].map( {'spam': 1, 'ham': 0} ).astype(int)
data['length'] = data['text'].apply(len)
data.head()

,label,text,spam,length
0,ham,"Go until jurong point, crazy.. Available only ...",0,111
1,ham,Ok lar... Joking wif u oni...,0,29
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1,155
3,ham,U dun say so early hor... U c already then say...,0,49
4,ham,"Nah I don't think he goes to usf, he lives aro...",0,61


### Distribution of the target variable

<font color='red'> 
    <h3>Aufgabe: Plotte die Verteilung der Labels </h3></font>

In [4]:
#Your code goes here

# **Part 2: Preprocessing**

**Damit wir das Datenset für klassifizierungs Algorithmen verwenden können muss dieses noch durch einige Schritte durch. Diese beinhalten die folgenden:**
* Remove Stop Words ("uninformative" Wörter ausschliessen
* Tokenization (Jedes Wort wird als "Token" angesehen)
* Vectorization (Text muss numerisch dargestellt werden)
* TF-IDF weighting  (Verhältnis zwischen Wort häufigkeit und Textlänge)


## 2.1 Remove Punctuation and Stopwords

### Punctuation
**Die folgende punctuation list wird für uns verwendet um den Text zu cleanen:**

In [5]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

### Stopwords  
Sklearn Dokumentation:  https://scikit-learn.org/stable/modules/feature_extraction.html#stop-words  

<font color='red'> 
    <h3>Aufgabe:</h3>
    </font>
Frage dich, wieso entfernt man stopwords aus Texten?
Was sind die Vorteile?
Könnte es auch Nachteile haben?


<font color='red'>Noch nicht weiter lesen, bis du dir Gedanken gemacht hast!</font>

#### Erklärung
Stopwords sind Wörter wie "und", "der", "ihn", die vermutlich keine Informationen zur Darstellung des Inhalts eines Textes liefern und entfernt werden können. Dies vermeidet, dass sie als Signal für Vorhersagen interpretiert werden. (Es gibt usecases, bei denen diese nützlich sein können. Beispiel: Zuordnung von Schreibstilen zu Persönlichkeiten.)

Wir verwenden hier die Stopwords der Library NLTK:

**NLTK**

In [31]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\luca7\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [32]:
from nltk.corpus import stopwords
stopwords.words("english")[100:110]

['here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each']

**Folgende Funktion entfernt die Stopwords und konvertiert den Text in Kleinschreibweise:**

In [8]:
def remove_punctuation_and_stopwords(sms):
    
    sms_no_punctuation = [ch for ch in sms if ch not in string.punctuation]
    sms_no_punctuation = "".join(sms_no_punctuation).split()
    
    sms_no_punctuation_no_stopwords = \
        [word.lower() for word in sms_no_punctuation if word.lower() not in stopwords.words("english")]
        
    return sms_no_punctuation_no_stopwords

In [9]:
data.head(5)

,label,text,spam,length
0,ham,"Go until jurong point, crazy.. Available only ...",0,111
1,ham,Ok lar... Joking wif u oni...,0,29
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1,155
3,ham,U dun say so early hor... U c already then say...,0,49
4,ham,"Nah I don't think he goes to usf, he lives aro...",0,61


In [10]:
data['text'].apply(remove_punctuation_and_stopwords).head()

0    [go, jurong, point, crazy, available, bugis, n...
1                       [ok, lar, joking, wif, u, oni]
2    [free, entry, 2, wkly, comp, win, fa, cup, fin...
3        [u, dun, say, early, hor, u, c, already, say]
4    [nah, dont, think, goes, usf, lives, around, t...
Name: text, dtype: object

In [11]:
data.head(5)

,label,text,spam,length
0,ham,"Go until jurong point, crazy.. Available only ...",0,111
1,ham,Ok lar... Joking wif u oni...,0,29
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1,155
3,ham,U dun say so early hor... U c already then say...,0,49
4,ham,"Nah I don't think he goes to usf, he lives aro...",0,61


## 2.2 Bag of words with CountVectorizer

### The Bag of Words representation

https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction  

Die Textanalyse ist ein wichtiger Anwendungsbereich für Algorithmen. Die Rohdaten, eine Folge von Symbolen, können jedoch nicht direkt in die Algorithmen eingespeist werden, da die meisten von ihnen numerische Merkmalsvektoren mit einer festen Größe erwarten und nicht die rohen Textdokumente mit variabler Länge.  
Um dieses Problem zu lösen, bietet scikit-learn Hilfsprogramme für die gebräuchlichsten Methoden zur Extraktion von numerischen Merkmalen aus Textinhalten, nämlich:

**Tokenization**  
Tokenisierung von Zeichenketten und Vergabe einer ganzzahligen ID für jedes mögliche Token, z. B. durch Verwendung von Leerzeichen und Satzzeichen als Tokentrennzeichen.  
**Vectorization**  
Zählen des Vorkommens von Token in jedem Dokument.
**TF-IDF**  
Normalisierung und Gewichtung mit abnehmender Bedeutung von Tokens, die in der Mehrzahl der Proben/Dokumente vorkommen. 

**Bag of Words**  
In diesem Schema werden die Merkmale und Stichproben wie folgt definiert:
Jede einzelne Token-Häufigkeit (normalisiert oder nicht) wird als Feature behandelt.  
Der Vektor aller Tokenhäufigkeiten für ein bestimmtes Dokument wird als multivariate Stichprobe betrachtet.  
Ein Korpus von Dokumenten kann somit durch eine Matrix mit einer Zeile pro Dokument und einer Spalte pro Token (z.B. Wort), das im Korpus vorkommt, dargestellt werden.  
Wir bezeichnen die Vektorisierung als den allgemeinen Prozess der Umwandlung einer Sammlung von Textdokumenten in numerische Merkmalsvektoren.  

Diese spezifische Strategie (Tokenisierung, Zählung und Normalisierung) wird als **Bag of Words** oder "Bag of n-grams"-Darstellung bezeichnet. 

Die Dokumente werden durch das Vorkommen von Wörtern beschrieben, wobei die Informationen über die relative Position der Wörter im Dokument vollständig ignoriert werden.

Weitere Informationenen:
https://en.wikipedia.org/wiki/Bag-of-words_model  
https://en.wikipedia.org/wiki/Document-term_matrix  

Einführung zu Bag-of-Words in NLP:  
https://medium.com/greyatom/an-introduction-to-bag-of-words-in-nlp-ac967d43b428

Wir verwenden hier den CountVectorizer from sklearn als BOW (Bag of Words) model.  
https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html    

Als Tokenizer verwenden wir die remove_punctuation_and_stopwords Funktion, die wir oben definiert haben.

In [12]:
data.head(5)

,label,text,spam,length
0,ham,"Go until jurong point, crazy.. Available only ...",0,111
1,ham,Ok lar... Joking wif u oni...,0,29
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1,155
3,ham,U dun say so early hor... U c already then say...,0,49
4,ham,"Nah I don't think he goes to usf, he lives aro...",0,61


In [13]:
from sklearn.feature_extraction.text import CountVectorizer
bow_transformer = CountVectorizer(analyzer = remove_punctuation_and_stopwords).fit(data['text'])

In [14]:
print(len(bow_transformer.vocabulary_))

9432


In allen SMS-Nachrichten zählte bow_transformer 9431 verschiedene Wörter.

### Applying bow_transformer on all messages

In [15]:
bow_data = bow_transformer.transform(data['text'])

In [16]:
bow_data.shape

(5572, 9432)

In [17]:
bow_data.nnz

49773

**Sparsity: percentage of none zero entries**  
https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction  

**Sparsity**  
Da die meisten Dokumente typischerweise eine sehr kleine Teilmenge der im Korpus verwendeten Wörter verwenden,  
wird die resultierende Matrix viele Features haben, die Nullen sind (typischerweise mehr als 99 %).  
Eine Sammlung von 10.000 kurzen Textdokumenten (z. B. E-Mails) verwendet beispielsweise ein Vokabular  
mit einer Größe in der Größenordnung von insgesamt 100.000 eindeutigen Wörtern, während jedes Dokument 100 bis   
1000 eindeutige Wörter verwendet.  
Um eine solche Matrix im Speicher speichern zu können, aber auch um algebraische Operationen zu beschleunigen, verwenden Matrix/Vektor,  
werden Implementierungen typischerweise eine spärliche Darstellung verwenden, wie sie im scipy.sparse-Paket verfügbar ist.


Anzahl nicht Null Elemente geteilt durch die Matrixgrösse

In [18]:
print( bow_data.nnz / (bow_data.shape[0] * bow_data.shape[1]) *100 )

0.09470631054216923


Ungefähr 10% der Matrix sind nicht Null (=ones)

## 2.3 Term frequency inverse document frequency - TFIDF

### From occurrences to frequencies  
https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html#from-occurrences-to-frequencies

Die Anzahl der Vorkommnisse ist ein guter Anfang, aber es gibt ein Problem: Längere Dokumente haben im Durchschnitt höhere Zählwerte als kürzere Dokumente, auch wenn sie über dieselben Themen sprechen.  
Um diese potenziellen Diskrepanzen zu vermeiden, reicht es aus, die Anzahl der Vorkommen jedes Worts in einem Dokument durch die Gesamtzahl der Wörter im Dokument zu teilen: Diese neuen Merkmale werden **tf für Term Frequencies** genannt.  

Eine weitere Verfeinerung von tf ist die Herabstufung der Gewichte für Wörter, die in vielen Dokumenten im Korpus vorkommen und daher weniger informativ sind als solche, die nur in einem Dokument vorkommen.  
daher weniger informativ sind als diejenigen, die nur in einem kleineren Teil des Korpus vorkommen.  
Diese Herabstufung wird **tf-idf für "Term Frequency times Inverse Document Frequency "** genannt.

Mehr Informationen:
https://en.wikipedia.org/wiki/Tf%E2%80%93idf

https://www.kaggle.com/adamschroeder/countvectorizer-tfidfvectorizer-predict-comments

### TfidfTransformer from sklearn
Sowohl tf als auch tf-idf können mit TfidfTransformer wie folgt berechnet werden:   
https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html

In [19]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer().fit(bow_data)

In [20]:
data_tfidf = tfidf_transformer.transform(bow_data)

In [21]:
data_tfidf

<5572x9432 sparse matrix of type '<class 'numpy.float64'>'
	with 49773 stored elements in Compressed Sparse Row format>

In [22]:
np.shape(data_tfidf)

(5572, 9432)

## Train Test split

Nur TFIDF matrix

In [23]:
from sklearn.model_selection import train_test_split

data_tfidf_train, data_tfidf_test, label_train, label_test = \
    train_test_split(data_tfidf, data["spam"], test_size=0.3, random_state=5)

Für TFIDF matrix and feature "length"

In [24]:
from scipy.sparse import  hstack
X2 = hstack((data_tfidf ,np.array(data['length'])[:,None])).A

In [25]:
X2_train, X2_test, y2_train, y2_test = \
    train_test_split(X2, data["spam"], test_size=0.3, random_state=5)

# Part 3: Classifiers

### Naive Bayes Classifier

sparse matrix to matrix

In [26]:
data_tfidf_train = data_tfidf_train.A
data_tfidf_test = data_tfidf_test.A

MNB Model mit TFIDF matrix

In [27]:
spam_detect_model = MultinomialNB().fit(data_tfidf_train, label_train)
pred_test_MNB = spam_detect_model.predict(data_tfidf_test)
acc_MNB = accuracy_score(label_test, pred_test_MNB)
print(acc_MNB)

0.965311004784689


Erster Classifier funktioniert schon nicht schlecht: 
Er hat eine Accuracy von 96.5 % für das Testset.

In [28]:
scaler = MinMaxScaler()
data_tfidf_train_sc = scaler.fit_transform(data_tfidf_train)
data_tfidf_test_sc  = scaler.transform(data_tfidf_test)

MNB Model mit TFIDF matrix, aber die Werte sind gescaled mit MinMaxScaler (Alles zwischen 0 und 1)

In [29]:
spam_detect_model_minmax = MultinomialNB().fit(data_tfidf_train_sc, label_train)
pred_test_MNB = spam_detect_model_minmax.predict(data_tfidf_test_sc)
acc_MNB = accuracy_score(label_test, pred_test_MNB)
print(acc_MNB)

0.9826555023923444


Die Anwendung des Min-Max-Skalierers auf die TFIDF-Matrix verbessert die Leistung des MNB-Classifiers:  
Er hat nun eine Accuracy von 98,2 % für das Testset. 

In [30]:
data_tfidf_train.shape

(3900, 9432)

<font color='red'><h1>Aufgabe:</h1></font>
### Kannst du ein Neuronales Netzwerk erstellen, welches die Texte klassifiziert?
### Findest du eine bessere Lösung?

### Kannst du deine eigene Texte klassifizieren lassen?